### 資料整理

In [20]:
import pandas as pd
#read xlsx file
#add row names
A_df = pd.read_excel('A.xlsx', header=None, names=['id', 'name',"en_name" ,'methods',"Nationality","class","mail"])
#remvoe methods == NaN
A_df = A_df.dropna(subset=['methods'])
A_df = A_df[['id','methods']]
#convert id to int
A_df['id'] = A_df['id'].astype(int)

B_df=pd.read_excel('B.xls', header=None)
#remove the first row,second row
B_df = B_df.drop([0,1])
#set the first row as the column names
B_df.columns = B_df.iloc[0]
#keep rows(期末考) to numeric,如果不是数字，转换为NaN
B_df['總成績'] = pd.to_numeric(B_df['總成績'], errors='coerce')
#只保留期末考不是NaN的行
B_df = B_df.dropna(subset=['總成績'])
#只留下前三行跟總成績
B_df = B_df[['學號','總成績']]
#rename columns
B_df.columns = ['id','score']
#merge two dataframes
B_df[['id']]=B_df[['id']].astype(int)
result = pd.merge(B_df, A_df, on='id')
#drop id column
result = result.drop(columns=['id'])
#sort by score
result = result.sort_values(by='score', ascending=False)
#add rank column
result['rank'] = result.rank(ascending=False,method='min')['score'].astype(int)
#add pass if score if(>=60) "Y" else NA
result['pass'] = result['score'].apply(lambda x: 'Y' if x>=60 else '')
#add top 30%
result['top30'] = result['rank'].apply(lambda x: 'Y' if x<=result.shape[0]*0.3 else '')
result=result.reset_index(drop=True)
# 改成 methods, score, rank, pass, top30
result = result[['methods','score','rank','pass','top30']]
result

,methods,score,rank,pass,top30
0,大學甄選入學個人申請,100.0,1,Y,Y
1,大學考試入學,100.0,1,Y,Y
2,大學考試入學,100.0,1,Y,Y
3,大學考試入學,100.0,1,Y,Y
4,大學考試入學,99.0,5,Y,Y
5,大學甄選入學繁星推薦,98.0,6,Y,Y
6,大學甄選入學個人申請,97.0,7,Y,Y
7,大學甄選入學繁星推薦,97.0,7,Y,Y
8,大學考試入學,97.0,7,Y,Y
9,大學考試入學,96.0,10,Y,Y


### 成績分析表

In [21]:
#建立df,col_names=["入學管道","該科及格人數","該科前30%人數","該科期末成績排名百分比"]
df = pd.DataFrame(columns=["入學管道","學生人數","該科及格人數比例","該科前30%人數比例","該科期末成績排名百分比例"])
entry_methods = ["大學考試入學","大學甄選入學個人申請","大學甄選入學繁星推薦","外國學生申請入學",
                 "寒假大學轉學考","僑生/個人申請","僑生分發/僑大先修班分發"]
for entry_method in entry_methods:
    #計算該科及格人數(非''的數量)
    pass_num = (result[result['methods']==entry_method]['pass']=='Y').sum()
    top30_num = (result[result['methods']==entry_method]['top30']=='Y').sum()
    students_num = result[result['methods']==entry_method].shape[0]
    #該科期末成績排名百分比例 算式=(該管道學生名次加總/該管道學生人數)/全班人數
    rank_percent = result[result['methods']==entry_method]['rank'].sum()/students_num/result.shape[0]
    new_row = pd.DataFrame({'入學管道': [entry_method], '該科及格人數比例': [pass_num/students_num], 
                            '該科前30%人數比例': [top30_num/students_num], 
                            '該科期末成績排名百分比例': [rank_percent], '學生人數': [students_num]})
    df = pd.concat([df, new_row], ignore_index=True)

#計算全班及格人數
pass_num = (result['pass']=='Y').sum()
students_num = result.shape[0]
new_row = pd.DataFrame({'入學管道': ['全班'], '該科及格人數比例': [pass_num/students_num], '學生人數': [students_num]})
df = pd.concat([df, new_row], ignore_index=True)
#convert to % if na is na
df['該科及格人數比例'] = df['該科及格人數比例'].apply(lambda x: '{:.2%}'.format(x) if not pd.isna(x) else x)
df['該科前30%人數比例'] = df['該科前30%人數比例'].apply(lambda x: '{:.2%}'.format(x) if not pd.isna(x) else x)
df['該科期末成績排名百分比例'] = df['該科期末成績排名百分比例'].apply(lambda x: '{:.2%}'.format(x) if not pd.isna(x) else x)
#save to csv
df.to_csv('成績分析表.csv', index=False, encoding='big5')

/tmp/ipykernel_127743/3471063662.py:15: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_127743/3471063662.py:11: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_127743/3471063662.py:12: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_127743/3471063662.py:13: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_127743/3471063662.py:11: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_127743/3471063662.py:12: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_127743/3471063662.py:13: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_127743/3471063662.py:11: RuntimeWarning:

i

### 全班原始成績

In [22]:
#copy the result to df1
df1=result.copy()
#rename columns
df1.columns = ['入學管道','分數','名次','及格','前30%']
#建立一個dataframe colsnames = [學期,課號,班別,課名], len(rows)=result.shape[0]
rows = result.shape[0]
columns = ['學期','課號','班別','課名']
df2 = pd.DataFrame(index=range(rows), columns=columns)
#concatenate two dataframes
df = pd.concat([df2,df1], axis=1)
#save to csv
df.to_csv('全班原始成績.csv', index=False,encoding='big5')

In [23]:
#基於entry_methods建立box plot(plotly)
entry_methods = ["大學考試入學","大學甄選入學個人申請","大學甄選入學繁星推薦","外國學生申請入學","寒假大學轉學考","僑生/個人申請","僑生分發/僑大先修班分發"]
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
for entry_method in entry_methods:
    fig.add_trace(go.Box(y=result[result['methods']==entry_method]['score'], name=entry_method))
fig.update_layout(title_text='各入學管道成績分布')
#save to html
fig.write_html('各入學管道成績分布.html')